<a href="https://colab.research.google.com/github/anand25695/coursera-DeepLearning.ai/blob/master/CrackDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
cd drive/My Drive/YE358311_Fender_apron

/content/drive/My Drive/YE358311_Fender_apron


In [0]:
ls

drive/  sample_data/


In [0]:
from PIL import Image
from numpy import array
import glob
import numpy as np
import matplotlib.pyplot as plt

In [0]:
images = []
for f in glob.iglob("YE358311_Healthy/*"):
    images.append(np.array(Image.open(f)))

images = np.array(images)

In [0]:
images.shape

(8, 3120, 4160, 3)

In [0]:
images_crack = []
for f in glob.iglob("YE358311_defects/YE358311_Crack_and_Wrinkle_defect/*"):
    images_crack.append(np.array(Image.open(f)))

images_crack = np.array(images_crack)

In [0]:
images_crack.shape

(12, 3120, 4160, 3)

In [0]:
y_normal= np.ones((len(images),1))
y_crack=np.zeros((len(images_crack),1))

In [0]:
y_normal.shape

(8, 1)

In [0]:
all_x= np.concatenate((images, images_crack), axis=0)
all_y =np.concatenate((y_normal, y_crack),axis = 0)

In [0]:
l = []
for i in range(len(all_x)):
    a = (all_x[i],all_y[i])
    l.append(a)

In [0]:
k=np.array(l)
k

array([[array([[[ 93,  74, 138],
        [114,  95, 159],
        [123, 104, 168],
        ...,
        [115,  95, 164],
        [120, 100, 169],
        [110,  90, 159]],

       [[ 90,  71, 135],
        [115,  96, 160],
        [118,  99, 163],
        ...,
        [121, 101, 170],
        [111,  91, 160],
        [110,  90, 159]],

       [[ 86,  67, 131],
        [ 98,  79, 143],
        [ 96,  77, 141],
        ...,
        [111,  90, 159],
        [107,  87, 156],
        [108,  89, 155]],

       ...,

       [[ 91,  73, 131],
        [ 92,  74, 132],
        [ 87,  69, 127],
        ...,
        [100,  80, 141],
        [ 87,  67, 128],
        [ 84,  64, 127]],

       [[ 91,  71, 130],
        [ 89,  71, 129],
        [ 89,  71, 129],
        ...,
        [ 97,  77, 138],
        [ 82,  62, 123],
        [ 86,  66, 127]],

       [[ 89,  69, 128],
        [ 92,  72, 131],
        [ 93,  75, 133],
        ...,
        [111,  91, 152],
        [ 94,  74, 135],
        [ 99,  7

In [0]:
import random
random.shuffle(k)

In [0]:
all_train= k[0:int(len(k)*0.8),]
all_test =k[0:(len(k)-int(len(k)*0.8)),]
x_train = np.stack(all_train[0:,0], axis=0)
y_train =np.stack(all_train[0:,1], axis=0)
x_test =np.stack(all_test[0:,0], axis=0)
y_test = np.stack(all_test[0:,1], axis= 0)              

In [0]:
y_train.shape

(16, 1)

In [0]:
input_shape = x_train.shape[1:]

In [0]:
from __future__ import print_function
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
import numpy as np
import os

Using TensorFlow backend.


In [0]:
batch_size = 10  # orignal paper was trained all networks with batch_size=128
epochs = 50
num_classes = 2

In [0]:
subtract_pixel_mean = False

In [0]:
n = 3
version = 1
if version == 1:
    depth = n * 6 + 2
elif version == 2:
    depth = n * 9 + 2

In [0]:
model_type = 'ResNet%dv%d' % (depth, version)

In [0]:
model_type

'ResNet20v1'

In [0]:
input_shape = x_train.shape[1:]

In [0]:
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

In [0]:
if subtract_pixel_mean:
    x_train_mean = np.mean(x_train, axis=0)
    x_train -= x_train_mean
    x_test -= x_train_mean

In [0]:
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print('y_train shape:', y_train.shape)

x_train shape: (16, 3120, 4160, 3)
16 train samples
4 test samples
y_train shape: (16, 1)


In [0]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
y_train

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.]], dtype=float32)

In [0]:
def lr_schedule(epoch):
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [0]:
def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x

In [0]:
def resnet_v1(input_shape, depth, num_classes=2):
    if (depth - 2) % 6 != 0:
        raise ValueError('depth should be 6n+2 (eg 20, 32, 44 in [a])')
    # Start model definition, we can change it according to our result or output.
    num_filters = 16
    num_res_blocks = int((depth - 2) / 6)

    inputs = Input(shape=input_shape)
    x = resnet_layer(inputs=inputs)
    # Instantiate the stack of residual units
    for stack in range(3):
        for res_block in range(num_res_blocks):
            strides = 1
            if stack > 0 and res_block == 0:  # first layer but not first stack
                strides = 16
            y = resnet_layer(inputs=x,
                             num_filters=num_filters,
                             strides=strides)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters,
                             activation=None)
            if stack > 0 and res_block == 0:  
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])
            x = Activation('relu')(x)
        num_filters *= 4

    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    model = Model(inputs=inputs, outputs=outputs)
    return model

In [0]:
def resnet_v2(input_shape, depth, num_classes=2):
    if (depth - 2) % 9 != 0:
        raise ValueError('depth should be 9n+2 (eg 56 or 110 in [b])')
    num_filters_in = 16
    num_res_blocks = int((depth - 2) / 9)

    inputs = Input(shape=input_shape)
    x = resnet_layer(inputs=inputs,
                     num_filters=num_filters_in,
                     conv_first=True)

    for stage in range(3):
        for res_block in range(num_res_blocks):
            activation = 'relu'
            batch_normalization = True
            strides = 1
            if stage == 0:
                num_filters_out = num_filters_in * 4
                if res_block == 0: 
                    activation = None
                    batch_normalization = False
            else:
                num_filters_out = num_filters_in * 2
                if res_block == 0:  
                    strides = 2   

            y = resnet_layer(inputs=x,
                             num_filters=num_filters_in,
                             kernel_size=1,
                             strides=strides,
                             activation=activation,
                             batch_normalization=batch_normalization,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_in,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_out,
                             kernel_size=1,
                             conv_first=False)
            if res_block == 0:

                x = resnet_layer(inputs=x,
                                 num_filters=num_filters_out,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])

        num_filters_in = num_filters_out

    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    model = Model(inputs=inputs, outputs=outputs)
    return model

In [0]:
if version == 2:
    model = resnet_v2(input_shape=input_shape, depth=depth)
else:
    model = resnet_v1(input_shape=input_shape, depth=depth)

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=lr_schedule(0)),
              metrics=['accuracy'])
model.summary()
print(model_type)

Learning rate:  0.001
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 3120, 4160, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 3120, 4160, 1 448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 3120, 4160, 1 64          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 3120, 4160, 1 0           batch_normalization_1[0][0]      
_______________________________________________________________________________________

In [0]:
lr_scheduler = LearningRateScheduler(lr_schedule)

In [0]:
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

In [0]:
model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)


Instructions for updating:
Use tf.cast instead.
Train on 16 samples, validate on 4 samples
Epoch 1/50


ResourceExhaustedError: ignored

In [0]:
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])